# Modelos de analise de sentimentos

In [4]:
from pysentimiento import create_analyzer

In [2]:
modelo = 'pysentimiento/bertweet-pt-sentiment'

In [7]:
modelo_analise_sentimento = create_analyzer(task='sentiment', lang='pt')

In [21]:
def analisar_sentimento(texto):
    resultado = modelo_analise_sentimento.predict(texto)
    return '\n'.join([f'Sentimento: {resultado.output}', 'Confiança: ', f'\tNegativo: {resultado.probas.get("NEG")}', f'\tNeutro: {resultado.probas.get("NEU")}', f'\tPositivo: {resultado.probas.get("POS")}'])

In [23]:
print(analisar_sentimento("Eu odeio quando a internet não funciona"))

Sentimento: NEG
Confiança: 
	Negativo: 0.9838698506355286
	Neutro: 0.009782686829566956
	Positivo: 0.006347368936985731


In [24]:
print(analisar_sentimento('Eu odeio este produto!'))

Sentimento: NEG
Confiança: 
	Negativo: 0.9920969009399414
	Neutro: 0.0034392480738461018
	Positivo: 0.004463882651180029


In [26]:
print(analisar_sentimento('Estes produtos foram vendidos fora da data de entrega'))

Sentimento: NEU
Confiança: 
	Negativo: 0.16409549117088318
	Neutro: 0.8187859654426575
	Positivo: 0.017118575051426888


In [28]:
print(analisar_sentimento('O melhor produtos de todos, mas tem suas resalvas devido fazer muito barulho.'))

Sentimento: NEU
Confiança: 
	Negativo: 0.15663763880729675
	Neutro: 0.5257287621498108
	Positivo: 0.31763359904289246
